In [4]:
import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [
    "ap_bus_details.csv", 
    "assam_bus_details.csv", 
    "chandigarh_bus_details.csv", 
    "himachal_bus_details.csv", 
    "kaac_bus_details.csv", 
    "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", 
    "sb_bus_details.csv", 
    "Telangana_bus_details.csv", 
    "up_bus_details.csv"
]   

# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Add ID column
id_column = pd.Series(range(1, len(combined_df) + 1), name='id')

# Concatenate the ID column with the combined DataFrame
combined_df = pd.concat([id_column, combined_df], axis=1)

# Save the updated DataFrame to a new CSV file
combined_df.to_csv("bus_routes.csv", index=False)

# Reload the updated CSV file (optional step)
df = pd.read_csv("bus_routes.csv")

# Replace 'INR ' with an empty string in the 'Price' column
df['Price'] = df['Price'].str.replace('INR ', '')

# Extract digits from 'Seat_Availability' column
df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')

# Drop rows with null values
df = df.dropna()

# Convert DataFrame to native Python types
df = df.astype(object).where(pd.notnull(df), None)

# Python MySQL connection
myconnection = pymysql.connect(
    host='127.0.0.1', 
    user='root', 
    passwd='WJ28@krhps', 
    database="redbus"
)

# Create SQL table schema by converting pandas dtypes to SQL types
a = ",".join(f"{i} {j}" for i,j in zip(df.columns,df.dtypes))\
    .replace("float64", "float")\
    .replace("object", "text")\
    .replace("int64", "int")

table_name = "bus_routes"

# Create the table in MySQL
cursor = myconnection.cursor()
cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({a})")

# Insert data into the table
for i in range(len(df)):
    row_values = tuple(df.iloc[i].to_list())
    cursor.execute(f"INSERT INTO {table_name} VALUES {row_values}")
    myconnection.commit()

# Close the connection
cursor.close()
myconnection.close()
